In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
from keras.applications.vgg19 import VGG19
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten

from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform

from keras import backend as K

import tensorflow as tf
import random
import os
from tqdm import tqdm
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import cv2
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

Using TensorFlow backend.


labels.csv
sample_submission.csv
test
train



In [2]:
modelPath = '../models/MNIST_opt/run_adam.h5';
os.environ["CUDA_VISIBLE_DEVICES"]="0"
a = keras.optimizers.Adam(lr=.4);
print(a.lr)

<tf.Variable 'Adam/lr:0' shape=() dtype=float32_ref>


In [3]:
def data():
    print('Getting data')
    df_train = pd.read_csv('../input/labels.csv')
    df_test = pd.read_csv('../input/sample_submission.csv')
    
    targets_series = pd.Series(df_train['breed'])
    one_hot = pd.get_dummies(targets_series, sparse = True)
    one_hot_labels = np.asarray(one_hot)
    
    im_size = 90
    x_train = []
    y_train = []
    x_test = []
    
    i = 0 
    for f, breed in tqdm(df_train.values[:10]):
        img = cv2.imread('../input/train/{}.jpg'.format(f))
        label = one_hot_labels[i]
        x_train.append(cv2.resize(img, (im_size, im_size)))
        y_train.append(label)
        i += 1
    
    y_train_raw = np.array(y_train, np.uint8)
    x_train_raw = np.array(x_train, np.float32) / 255.
    num_class = y_train_raw.shape[1]
    
    print('Splitting into training/validation')
    X_train, X_valid, Y_train, Y_valid = train_test_split(x_train_raw, y_train_raw, test_size=0.3, random_state=1)
    
    return X_train, Y_train, X_valid, Y_valid

In [4]:
# prepare data and model for hyperas

def model(X_train,Y_train,X_valid,Y_valid):
    print('Creating model')
     #with tf.device('/device:GPU:0'):
    base_model = VGG19(weights = 'imagenet',
                       include_top=False,
                       input_shape=(im_size, im_size, 3))

    # Add a new top layer
    model = Sequential()

    model.add(Conv2D(32, (3, 3), input_shape=(im_size,im_size,3)))
    model.add(Activation('relu'))
    BatchNormalization(axis=-1)
    model.add(Conv2D(32, (3, 3),))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
 
    BatchNormalization(axis=-1)
    model.add(Conv2D(64,(3, 3)))
    model.add(Activation('relu'))
    BatchNormalization(axis=-1)
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Flatten())
    # Fully connected layer

    BatchNormalization()
    model.add(Dense(512))
    model.add(Activation('relu'))
    BatchNormalization()
    model.add(Dropout(0.2))
    model.add(Dense(120))
    #model.add(Conv2D(10,(3,3)))
    #model.add(GlobalAveragePooling2D('none'))
    model.add(Activation('softmax'))
    #predictions = Dense(num_class, activation='softmax')(x)

    # This is the model we will train
    model = Model(inputs=base_model.input, outputs=predictions)

    # First: train only the top layers (which were randomly initialized)
    for i in range(len(base_model.layers)):
        base_model.layers[i].trainable = False

    # optimization
    lr={{uniform(0.00001,0.01)}};
    beta_1={{uniform(0.8,0.999)}};
    beta_2={{uniform(0.99,0.99999)}};
    decay={{uniform(0,0.01)}};

    print()
    print('lr=',lr);
    print('beta_1=',beta_1);
    print('beta_2=',beta_2);
    print('decay=',decay);
    print()

    adam = keras.optimizers.Adam(lr=lr,beta_1=beta_1,beta_2=beta_2,decay=decay)
    model.compile(loss='categorical_crossentropy', 
                  optimizer=adam, 
                  metrics=['accuracy'])

    callbacks_list = [];
    callbacks_list.append(keras.callbacks.EarlyStopping(
        monitor='val_acc',
        patience=10,
        verbose=1));

    print('Tensorboard NOT activated')

    model.fit(X_train, Y_train,
      epochs=100,
      batch_size = 64,
      validation_data=(X_valid, Y_valid),
      verbose=1,
      callbacks=callbacks_list)

    score, acc = model.evaluate(X_valid, Y_valid, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}


In [6]:
best_run, best_model = optim.minimize(model=model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='MNIST_optimized')

>>> Imports:
#coding=utf-8

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import keras
except:
    pass

try:
    from keras.applications.vgg19 import VGG19
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.layers import Dense, Dropout, Flatten
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform
except:
    pass

try:
    from keras import backend as K
except:
    pass

try:
    import tensorflow as tf
except:
    pass

try:
    import random
except:
    pass

try:
    import os
except:
    pass

try:
    from tqdm import tqdm
except:
    pass

try:
    from sklearn import preprocessing
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    import cv2
except:
    pass

try:
    from subpr

TypeError: 'generator' object has no attribute '__getitem__'

In [ ]:
X_train, Y_train, X_test, Y_test = data()
print('data loaded')

val_loss, val_acc = best_model.evaluate(X_test, Y_test);
print("Evalutation of best performing model:")
print("Validation loss: ", val_loss)
print("Validation accuracy: ", val_acc)

In [ ]:
print(best_model.weights())

In [ ]:
best_model.save(modelPath);
best_model.summary()